# Naural Networks

In [55]:
import pandas as pd
import numpy as np
from datetime import datetime
import keras
import tensorflow
from sklearn.preprocessing import OneHotEncoder

komentarz

In [125]:
chess = pd.read_csv("Data/games.csv")
chess = chess.loc[:1999, :]
chess.drop(columns=['created_at', 'last_move_at', 'victory_status', 'white_id', 'black_id', 'moves', 'opening_eco', 'opening_name', 'id'], inplace=True)
chess.head()

,rated,turns,winner,increment_code,white_rating,black_rating,opening_ply
0,False,13,white,15+2,1500,1191,5
1,True,16,black,5+10,1322,1261,4
2,True,61,white,5+10,1496,1500,3
3,True,61,white,20+0,1439,1454,3
4,True,95,white,30+3,1523,1469,5


In [126]:
chess.loc[:, 'rated'] = chess.loc[:, 'rated']*1

Zakodowanie wartości kategorycznych

In [127]:
temp = pd.get_dummies(chess.loc[:, ['increment_code', 'winner']])
chess.drop(columns=['increment_code', 'winner'], inplace=True)

Mozna przewidywac na poczatku bialy/remis/czarny a potem tylko czy biały wygra

In [128]:
chess = pd.concat([chess, temp], axis=1)
chess.replace({np.nan, 0})

,rated,turns,white_rating,black_rating,opening_ply,increment_code_10+0,increment_code_10+1,increment_code_10+10,increment_code_10+15,increment_code_10+2,...,increment_code_9+0,increment_code_9+1,increment_code_9+10,increment_code_9+2,increment_code_9+20,increment_code_9+3,increment_code_90+120,winner_black,winner_draw,winner_white
0,0,13,1500,1191,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,16,1322,1261,4,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,61,1496,1500,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,61,1439,1454,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,95,1523,1469,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,138,1537,1500,3,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1996,0,68,1500,1537,6,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1997,0,125,1500,1619,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1998,1,84,1026,1864,4,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [130]:
chess.loc[:, ['rated', 'turns', 'white_rating', 'black_rating', 'opening_ply']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   rated         2000 non-null   int64
 1   turns         2000 non-null   int64
 2   white_rating  2000 non-null   int64
 3   black_rating  2000 non-null   int64
 4   opening_ply   2000 non-null   int64
dtypes: int64(5)
memory usage: 78.2 KB


## Model 